## Steps

1. Load the houseprices data from Thinkful's database.
2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.
3. Now, split your data into train and test sets where 20% of the data resides in the test set.
4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!
5. Evaluate your best model on the test set.
6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.

**Step 1. Load the houseprices data from Thinkful's database.**

In [1]:
# Load the houseprices data from Thinkful's database.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'  
postgres_pw = '7*.8G9QH21'  
postgres_host = '142.93.121.174'  
postgres_port = '5432'  
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

hp_df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

hp_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


**Step 2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.**

The features were selected by this analysis. Refer to this [GitHub Pages](https://github.com/feelopk/Thinkful/blob/master/M19.3_Meeting%20the%20assumptions%20of%20linear%20regression_Assignment%202%20(House%20prices).ipynb).

In [2]:
hp_sel = hp_df[['saleprice', 'lotfrontage', 'lotarea', 'masvnrarea', 'bsmtfinsf1', 
                'totalbsmtsf', 'firstflrsf', 'secondflrsf', 'grlivarea', 'garagearea']]
hp_sel.replace(0, 1, inplace=True)
hp_sel.fillna(1, inplace=True)
hp_sellog = np.log(hp_sel)
hp_sellog_picked = hp_sellog[['saleprice', 'firstflrsf', 'grlivarea']]

**Step 3. Now, split your data into train and test sets where 20% of the data resides in the test set.**

In [3]:
X = hp_sellog_picked[['firstflrsf', 'grlivarea']]
Y = hp_sellog_picked['saleprice']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("The number of observations in the training set is {}".format(X_train.shape[0]))
print("The number of observations in the test set is {}".format(X_test.shape[0]))

The number of observations in the training set is 1168
The number of observations in the test set is 292


**Step 4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!**

1. Let's start with OLS regression.

In [13]:
from sklearn import linear_model
lrm = linear_model.LinearRegression()
lrm.fit(X_train, y_train)

lrm_y_train = lrm.predict(X_train)
lrm_y_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)), '\n')
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, lrm_y_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, lrm_y_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, lrm_y_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - lrm_y_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.5830986251228254 

-----Test set statistics-----
R-squared of the model in the test set is: 0.6336389874815189
Mean absolute error of the prediction is: 0.19571141237787565
Mean squared error of the prediction is: 0.06708278197141096
Root mean squared error of the prediction is: 0.25900344007640314
Mean absolute percentage error of the prediction is: 1.6423068988146043


2. This time, I will do Ridge Regression with built in cross-validation method.

In [15]:
alphas = [np.power(10.0, p) for p in np.arange(-10, 40, 1)]

rgm_cv = linear_model.RidgeCV(alphas=alphas, cv=5)
rgm_cv.fit(X_train, y_train)

rgm_y_train = rgm_cv.predict(X_train)
rgm_y_test = rgm_cv.predict(X_test)

print("Best alpha value is: {}".format(rgm_cv.alpha_))
print("R-squared of the model in the training set is: {}".format(rgm_cv.score(X_train, y_train)), '\n')
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(rgm_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, rgm_y_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, rgm_y_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, rgm_y_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - rgm_y_test) / y_test)) * 100))

Best alpha value is: 1.0
R-squared of the model in the training set is: 0.5830787509561941 

-----Test set statistics-----
R-squared of the model in the test set is: 0.6331752844249631
Mean absolute error of the prediction is: 0.19572859989654126
Mean squared error of the prediction is: 0.06716768863445514
Root mean squared error of the prediction is: 0.2591672985437305
Mean absolute percentage error of the prediction is: 1.6423620289718905


3. This time, I will do Lasso Regression with built in cross-validation method.

In [16]:
lass_cv = linear_model.LassoCV(alphas=alphas, cv=5)
lass_cv.fit(X_train, y_train)

lass_y_train = lass_cv.predict(X_train)
lass_y_test = lass_cv.predict(X_test)

print("Best alpha value is: {}".format(lass_cv.alpha_))
print("R-squared of the model in the training set is: {}".format(lass_cv.score(X_train, y_train)), '\n')
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lass_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, lass_y_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, lass_y_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, lass_y_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - lass_y_test) / y_test)) * 100))

Best alpha value is: 1e-05
R-squared of the model in the training set is: 0.5830986178623121 

-----Test set statistics-----
R-squared of the model in the test set is: 0.6336281381775515
Mean absolute error of the prediction is: 0.19571225586548768
Mean squared error of the prediction is: 0.06708476854058106
Root mean squared error of the prediction is: 0.25900727507269183
Mean absolute percentage error of the prediction is: 1.6423122948800752


4. This time, I will do ElasticNet Regression with built in cross-validation method.

In [17]:
elanet_cv = linear_model.ElasticNetCV(alphas=alphas, cv=5)
elanet_cv.fit(X_train, y_train)

elanet_y_train = elanet_cv.predict(X_train)
elanet_y_test = elanet_cv.predict(X_test)

print("Best alpha value is: {}".format(elanet_cv.alpha_))
print("R-squared of the model in the training set is: {}".format(elanet_cv.score(X_train, y_train)), '\n')
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(elanet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, elanet_y_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, elanet_y_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, elanet_y_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - elanet_y_test) / y_test)) * 100))

Best alpha value is: 0.0001
R-squared of the model in the training set is: 0.5830981507304196 

-----Test set statistics-----
R-squared of the model in the test set is: 0.6335540250491034
Mean absolute error of the prediction is: 0.1957172179351988
Mean squared error of the prediction is: 0.0670983390752914
Root mean squared error of the prediction is: 0.25903347095557244
Mean absolute percentage error of the prediction is: 1.642341766621318


**Step 5. Evaluate your best model on the test set.**

In [19]:
# Let's make table to look around at a glance. 

OLS = [0, 0.5830986251228254, 0.6336389874815189, 0.19571141237787565, 0.06708278197141096, 0.25900344007640314, 1.6423068988146043]
Ridge = [1.0, 0.5830787509561941 , 0.6331752844249631, 0.19572859989654126, 0.06716768863445514, 0.2591672985437305, 1.6423620289718905]
Lasso = [1e-05, 0.5830986178623121 , 0.6336281381775515, 0.19571225586548768, 0.06708476854058106, 0.25900727507269183, 1.6423122948800752]
ElasticNet = [0.0001, 0.5830981507304196, 0.6335540250491034, 0.1957172179351988, 0.0670983390752914, 0.25903347095557244, 1.642341766621318]

data = [OLS, Ridge, Lasso, ElasticNet]
index_name = ['OLS', 'Ridge', 'Lasso', 'ElasticNet']
col_name = ['Best Alpha', 'R-square (Training)', 'R-square (Test)', 'MAE', 'MSE', 'RMSE', 'MAPE']
table = pd.DataFrame(data, index=index_name, columns=col_name)
table

,Best Alpha,R-square (Training),R-square (Test),MAE,MSE,RMSE,MAPE
OLS,0.00000,0.583099,0.633639,0.195711,0.067083,0.259003,1.642307
Ridge,1.00000,0.583079,0.633175,0.195729,0.067168,0.259167,1.642362
Lasso,0.00001,0.583099,0.633628,0.195712,0.067085,0.259007,1.642312
ElasticNet,0.00010,0.583098,0.633554,0.195717,0.067098,0.259033,1.642342


In [20]:
table.describe()

,Best Alpha,R-square (Training),R-square (Test),MAE,MSE,RMSE,MAPE
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.250028,0.583094,0.633499,0.195717,0.067108,0.259053,1.642331
std,0.499982,0.000010,0.000219,0.000008,0.000040,0.000077,0.000026
min,0.000000,0.583079,0.633175,0.195711,0.067083,0.259003,1.642307
25%,0.000008,0.583093,0.633459,0.195712,0.067084,0.259006,1.642311
50%,0.000055,0.583098,0.633591,0.195715,0.067092,0.259020,1.642327
75%,0.250075,0.583099,0.633631,0.195720,0.067116,0.259067,1.642347
max,1.000000,0.583099,0.633639,0.195729,0.067168,0.259167,1.642362


When you look into the data table, the goodness model would have the largest R-squared value of Test set. And the rest values must be smaller than any others.  
**Then, the OLS model is the best model for this data set which is well presenting the outcome.**

**Step 6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.**

1. useful factors that are not included in the dataset.
 - Interest Rate
 - Crime Rate
 - Neighboring school ranking
 - NASDAQ points
 - New housing supply amount
 - Marriage rate


2. Implications of adding these external variables into your model.
If the model is too complex, in other words, if there are too many features, it would be a possiblity to make the model overfitted. Moreover, if the newly added features have some noises, I mean, not quite related to the outcome, then it could be a reason that the model overfitted.  
To reduce this kinds of matters, we need to collect the features in the process to figure out if those are qute related to the outcome or not.

